In [1]:
import pandas as pd

# Filter out the training set (is_correct_on(gpt) and !is_correct_on(llama))
### 일단 T에 대해서만

In [11]:
training_set = set()
gpt_df = pd.read_csv('/home/yl3427/cylab/selfCorrectionAgent/result/1112_t14_gpt_test_0_outof_8runs.csv').sort_values(by="patient_filename")
llama_df = pd.read_csv('/home/yl3427/cylab/selfCorrectionAgent/result/1208_t14_llama3_kewltm_0_outof_10runs.csv').sort_values(by="patient_filename")
for i in range(len(gpt_df)):
    label = gpt_df.loc[i, 't']
    gpt = gpt_df.loc[i, 'gpt4o_t_stage']
    llama = llama_df.loc[i, 't14_kewltm_t_pred']
    if (gpt != llama) and (gpt == f"T{label+1}"):
        training_set.add(gpt_df.loc[i, 'patient_filename'])

2 T3 T2
1 T2 T1
1 T2 T4
2 T3 T2
0 T1 T2
2 T3 T2
0 T1 T2
0 T1 T2
0 T1 T2
2 T3 T2
2 T3 T4
0 T1 T2
0 T1 T2
0 T1 T2
2 T3 T2
1 T2 T4
0 T1 T2
1 T2 T4
2 T3 T2
2 T3 T2
1 T2 T1
0 T1 T2
1 T2 T4
1 T2 T1
1 T2 T4
2 T3 T4
1 T2 T3
2 T3 T2
0 T1 T2
0 T1 T2
1 T2 T4
2 T3 T2
1 T2 T4
0 T1 T2
1 T2 T1
1 T2 T1
0 T1 T2
2 T3 T2
0 T1 T2
2 T3 T2
0 T1 T2
2 T3 T4
0 T1 T2
1 T2 T4
2 T3 T4
2 T3 T4
2 T3 T2
0 T1 T2
2 T3 T4
0 T1 T2
2 T3 T2
1 T2 T4
0 T1 T2
2 T3 T2
0 T1 T2
0 T1 T2
2 T3 T4
2 T3 T4
0 T1 T2
0 T1 T2
1 T2 T1
0 T1 T2
1 T2 T1
0 T1 T4
2 T3 T2
2 T3 T2
1 T2 T1
0 T1 T2
2 T3 T2
1 T2 T1
2 T3 T2
0 T1 T4
